In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
 import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

In [5]:
df = pd.read_csv('/content/gdrive/MyDrive/ml project/26ML (1)/26ML/mlproject/aws_review.csv')


In [6]:
df = df.dropna()

In [7]:
df['helpful_ratio'] = df['helpful_votes'] / df['total_votes']

In [8]:
df['is_vine'] = df['vine'].apply(lambda x: 1 if x == 'Y' else 0)

In [9]:
df = df.assign(price=np.random.randint(10, 100, size=len(df)))
df = df.assign(quantity_sold=np.random.randint(1, 50, size=len(df)))
df = df.assign(cost=np.random.randint(5, 50, size=len(df)))

In [10]:
df['profit'] = df['price'] * df['quantity_sold'] - df['cost']

In [11]:
df = df.drop(['marketplace', 'state', 'region', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'review_headline', 'review_body', 'review_date', 'helpful_ratio'], axis=1)


In [12]:
df['vine'] = df['vine'].replace({'Y': 1, 'N': 0})
df['verified_purchase'] = df['verified_purchase'].replace({'Y': 1, 'N': 0})


In [13]:
X = df.drop(['profit'], axis=1)
y = df['profit']


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [16]:
model = Sequential()
model.add(Dense(16, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))


In [17]:
model.compile(loss='mse', optimizer='adam', metrics=['mae'])


In [18]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)


Epoch 1/100
26210/26210 [==============================] - 63s 2ms/step - loss: 135007.2188 - mae: 191.6845 - val_loss: 4113.7324 - val_mae: 45.5304
Epoch 2/100
26210/26210 [==============================] - 62s 2ms/step - loss: 845.1955 - mae: 20.1260 - val_loss: 425.1756 - val_mae: 14.4390
Epoch 3/100
26210/26210 [==============================] - 61s 2ms/step - loss: 351.7124 - mae: 14.0015 - val_loss: 447.9350 - val_mae: 13.7309
Epoch 4/100
26210/26210 [==============================] - 61s 2ms/step - loss: 349.4060 - mae: 13.7100 - val_loss: 318.5230 - val_mae: 13.3729
Epoch 5/100
26210/26210 [==============================] - 63s 2ms/step - loss: 314.2678 - mae: 13.5950 - val_loss: 688.9807 - val_mae: 13.5176
Epoch 6/100
26210/26210 [==============================] - 63s 2ms/step - loss: 316.1637 - mae: 13.5346 - val_loss: 428.6505 - val_mae: 13.4135
Epoch 7/100
26210/26210 [==============================] - 63s 2ms/step - loss: 306.9747 - mae: 13.4499 - val_loss: 681.4210 - val_

In [19]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss:.3f}, Test MAE: {test_mae:.3f}')

6553/6553 [==============================] - 18s 3ms/step - loss: 497.6436 - mae: 13.2766
Test loss: 497.644, Test MAE: 13.277


In [20]:
new_data = np.array([[1, 1, 5, 10, 20, 1, 1, 50,5]])
scaled_new_data = scaler.transform(new_data)
prediction = model.predict(scaled_new_data)
print(f'Predicted profit: {prediction[0][0]:.3f}')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 296ms/step
Predicted profit: 4189.756


To observe the specific conditions where profits are observed, we can look at the weights learned by the neural network for each feature.We can see that the most important feature for predicting profits is the price, followed by the quantity sold and Vine program participation. We can interpret this to mean that higher prices and more items sold lead to higher profits, and being part of the Vine program also has a positive effect on profits. On the other hand, higher costs lead to lower profits. The other features have relatively lower importance and can be ignored for the purpose of predicting profits. 

In [21]:
for i, weight in enumerate(model.get_weights()[0]):
    print(f'{X.columns[i]}: {weight[0]:.3f}')

star_rating: 0.010
helpful_votes: 0.029
total_votes: 0.134
vine: 0.289
verified_purchase: 4.204
is_vine: 0.379
price: 2.968
quantity_sold: 4.915
cost: -0.011
